In [21]:
import os

train_files = [os.path.join("data/icons/outline/train", f) for f in os.listdir("data/icons/outline/train")]

In [32]:
# Create file with content of all training files
with open("data/icons/train.txt", "w") as outfile:
    for fname in train_files:
        with open(fname) as infile:
            outfile.write(infile.read())
            outfile.write("\n")

In [23]:
import tokenizers

In [24]:
model = tokenizers.models.BPE()
trainer = tokenizers.trainers.BpeTrainer(vocab_size=256, special_tokens=["[PAD]", "[SOS]", "[EOS]", "[MASK]", "[UNK]"])

In [13]:
tokenizer = tokenizers.Tokenizer(model=model)
tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.Sequence(
    [tokenizers.pre_tokenizers.Digits(True)]
)

tokenizer.add_tokens(
    *[
        [tokenizers.AddedToken(chr(i)) for i in range(32, 127)]
        + [
            tokenizers.AddedToken(
                '''
  <path d="'''
            ),
            tokenizers.AddedToken('" />'),
            tokenizers.AddedToken('\n'),
            tokenizers.AddedToken('''
<svg
  xmlns="http://www.w3.org/2000/svg"
  width="24"
  height="24"
  viewBox="0 0 24 24"
  fill="none"
  stroke="currentColor"
  stroke-width="2"
  stroke-linecap="round"
  stroke-linejoin="round"
>'''),
            tokenizers.AddedToken('</svg>'),
        ]
    ]
)

tokenizer.train(["data/icons/train.txt"], trainer=trainer)
print(tokenizer.get_vocab_size())




106


In [20]:
viz = EncodingVisualizer(tokenizer)
viz(icons[0:1024])

In [16]:
with open("data/icons/train.txt") as f:
    icons = f.read()
encoded = tokenizer.encode(icons)

In [18]:
print(len(icons))
print(encoded)

1912747
Encoding(num_tokens=955970, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [19]:
from tokenizers.tools import EncodingVisualizer

In [ ]:
encoded = encoded.ids
# most frequent ids
frequency = {
    tokenizer.decode([i]): encoded.count(i)
    for i in sorted(set(encoded), key=lambda x: encoded.count(x), reverse=False)
}

print(frequency)
print(len(frequency))

In [ ]:
#plot distribution
import matplotlib.pyplot as plt
plt.bar(frequency.keys(), frequency.values())
plt.show()

In [ ]:
tokenizer.save("data/icons/vocab/tokenizer.json")